In [1]:
import pandas as pd
import xarray as xr

In [26]:
ds_temp = xr.open_dataset("bcn.grib", engine="cfgrib", 
                          backend_kwargs={"filter_by_keys": {"shortName": "2t"}})

# Leer solo tp (precipitación total)
ds_prec = xr.open_dataset("bcn.grib", engine="cfgrib", 
                          backend_kwargs={"filter_by_keys": {"shortName": "tp"}})



Ignoring index file 'bcn.grib.47d85.idx' older than GRIB file
C:\Users\rubas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
Ignoring index file 'bcn.grib.47d85.idx' older than GRIB file
C:\Users\rubas\AppData\Local\Packages\PythonSoftwareFoun

In [30]:
df_temp = ds_temp.to_dataframe().reset_index()
df_prec = ds_prec.to_dataframe().reset_index()

In [31]:
df_temp

,time,latitude,longitude,number,step,surface,valid_time,t2m
0,2019-01-01 00:00:00,41.3,2.1,0,0 days,0.0,2019-01-01 00:00:00,282.796143
1,2019-01-01 06:00:00,41.3,2.1,0,0 days,0.0,2019-01-01 06:00:00,282.968506
2,2019-01-01 12:00:00,41.3,2.1,0,0 days,0.0,2019-01-01 12:00:00,287.306641
3,2019-01-01 18:00:00,41.3,2.1,0,0 days,0.0,2019-01-01 18:00:00,285.851074
4,2019-01-02 00:00:00,41.3,2.1,0,0 days,0.0,2019-01-02 00:00:00,282.264893
...,...,...,...,...,...,...,...,...
9415,2025-06-11 18:00:00,41.3,2.1,0,0 days,0.0,2025-06-11 18:00:00,297.439697
9416,2025-06-12 00:00:00,41.3,2.1,0,0 days,0.0,2025-06-12 00:00:00,295.605469
9417,2025-06-12 06:00:00,41.3,2.1,0,0 days,0.0,2025-06-12 06:00:00,295.270996
9418,2025-06-12 12:00:00,41.3,2.1,0,0 days,0.0,2025-06-12 12:00:00,297.957764


In [32]:
valid_temp_no_prec = set(df_temp["valid_time"].unique()) - set(df_prec["valid_time"].unique())

# Valores únicos que están en prec pero no en temp
valid_prec_no_temp = set(df_prec["valid_time"].unique()) - set(df_temp["valid_time"].unique())

print(valid_prec_no_temp)
print(valid_temp_no_prec)


# Filtrar df_temp eliminando esos valid_time que no están en prec
df_temp = df_temp[~df_temp["valid_time"].isin(valid_temp_no_prec)]

# Filtrar df_prec eliminando esos valid_time que no están en temp
df_prec = df_prec[~df_prec["valid_time"].isin(valid_prec_no_temp)]

set()
set()


In [33]:
df_temp_reduced = df_temp[["valid_time", "latitude", "longitude", "t2m"]]
df_prec_reduced = df_prec[["valid_time", "latitude", "longitude", "tp"]]
print(len(df_prec_reduced))
print(len(df_temp_reduced))

9420
9420


In [34]:
df_merged = pd.merge(df_temp_reduced, df_prec_reduced, on=["valid_time", "latitude", "longitude"], how="inner")
df_merged

,valid_time,latitude,longitude,t2m,tp
0,2019-01-01 00:00:00,41.3,2.1,282.796143,0.000000e+00
1,2019-01-01 06:00:00,41.3,2.1,282.968506,0.000000e+00
2,2019-01-01 12:00:00,41.3,2.1,287.306641,0.000000e+00
3,2019-01-01 18:00:00,41.3,2.1,285.851074,0.000000e+00
4,2019-01-02 00:00:00,41.3,2.1,282.264893,1.904875e-07
...,...,...,...,...,...
9415,2025-06-11 18:00:00,41.3,2.1,297.439697,0.000000e+00
9416,2025-06-12 00:00:00,41.3,2.1,295.605469,0.000000e+00
9417,2025-06-12 06:00:00,41.3,2.1,295.270996,0.000000e+00
9418,2025-06-12 12:00:00,41.3,2.1,297.957764,0.000000e+00


In [35]:
df_merged.to_parquet("bcn_temp_prec.parquet", index=False)